# 1. ZenML Basics

## Default Local Stack

In [ ]:
from zenml.client import Client

client = Client()
client.activate_stack("default")

In [ ]:
from zenml import pipeline, step


@step
def my_step():
    print("Hello world!")


@pipeline
def my_pipeline():
    my_step()


my_pipeline()

In [ ]:
@step
def some_other_step():
    print("This is another step!")


some_other_step()

## Kubernetes Stack

In [ ]:
from zenml.client import Client

client = Client()
client.activate_stack("az_k8s_pigeon")

In [ ]:
from zenml import pipeline, step
from settings import docker_settings, kubernetes_settings

@step(settings={"orchestrator.kubernetes": kubernetes_settings})
def k8s_step():
    print("Training model...")


@pipeline(settings={"docker": docker_settings})
def k8s_pipeline():
    k8s_step()


k8s_pipeline()

In [ ]:
from settings import docker_settings, kubernetes_settings
from zenml import step


@step(
    settings={"orchestrator": kubernetes_settings, "docker": docker_settings}
)
def k8s_step():
    print("Training model...")


In [ ]:
k8s_step()

# 2. Let's get my cat ready for GenAI!

## Resize our images

In [17]:
# resize the images so that they're maximum 1000 pixels on the longest side
import os
import cv2

data_dir = "/home/strickvl/coding/zenml-projects/mlops-community-demo/data/cats_mixed"

target_dir = "/home/strickvl/coding/zenml-projects/mlops-community-demo/data/cats_resized"

os.makedirs(target_dir, exist_ok=True)

for filename in os.listdir(data_dir):
    if filename.endswith((".jpeg", ".jpg", ".png")):
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)
        height, width = image.shape[:2]
        
        # Calculate the scaling factor
        scale = min(1000 / width, 1000 / height)
        
        # Calculate new dimensions while maintaining aspect ratio
        new_width = int(width * scale)
        new_height = int(height * scale)
        
        # Resize the image
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
        
        cv2.imwrite(os.path.join(target_dir, filename), resized_image)
        # print(f"Resized image: {filename}")

print("Resizing process completed.")

Resizing process completed.


In [19]:
import os
from tabulate import tabulate

resized_dir = "/home/strickvl/coding/zenml-projects/mlops-community-demo/data/cats_resized"

image_data = []

for filename in os.listdir(resized_dir):
    if filename.endswith((".jpeg", ".jpg", ".png")):
        image_path = os.path.join(resized_dir, filename)
        file_size = os.path.getsize(image_path)  # Size in bytes
        file_size_kb = file_size / 1024  # Convert bytes to kilobytes
        image_data.append([filename, f"{file_size_kb:.2f} KB"])

# Print the table
print(tabulate(image_data[:5], headers=["Image Name", "Size"], tablefmt="plain", colalign=("left", "left")))

Image Name      Size
blupus-08.jpeg  206.30 KB
aria-06.jpeg    110.77 KB
blupus-01.jpeg  231.38 KB
aria-01.jpeg    220.62 KB
blupus-11.jpeg  251.65 KB


## (Local) Data Annotation

## Data Augmentation with Albumentations

Using the Albumentations library to make a variety of transformations for our
images to augment our dataset a little. We'll avoid things that crop the image,
but we'll do things like change the brightness, contrast, and saturation. We'll
also do things like add noise, blur, and sharpen.

In [8]:
import albumentations as A
import cv2

transform = A.Compose([
    A.RandomBrightnessContrast(p=0.2),
    A.RandomGamma(p=0.2),
    A.RandomShadow(p=0.2),
    A.RandomRain(p=0.2),
    A.RandomSnow(p=0.2),
])

In [9]:
import os
import numpy as np
from PIL import Image

data_dir = "/home/strickvl/coding/zenml-projects/mlops-community-demo/data/cats_mixed"
target_dir = "/home/strickvl/coding/zenml-projects/mlops-community-demo/data/blupus"

# Ensure the target directory exists
os.makedirs(target_dir, exist_ok=True)

for filename in os.listdir(data_dir):
    if filename.endswith((".jpeg", ".jpg", ".png")):
        image_path = os.path.join(data_dir, filename)
        try:
            image = Image.open(image_path)
            image_np = np.array(image)
            
            # Apply the transformation
            transformed = transform(image=image_np)
            transformed_image = transformed["image"]
            
            # Convert back to PIL Image
            transformed_pil = Image.fromarray(transformed_image)
            
            # Save the augmented image
            augmented_filename = f"augmented_{filename}"
            augmented_path = os.path.join(target_dir, augmented_filename)
            transformed_pil.save(augmented_path)
            
            print(f"Saved augmented image: {augmented_filename}")
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

print("Augmentation process completed.")


Saved augmented image: augmented_blupus-08.jpeg
Saved augmented image: augmented_aria-06.jpeg
Saved augmented image: augmented_blupus-01.jpeg
Saved augmented image: augmented_aria-01.jpeg
Saved augmented image: augmented_blupus-11.jpeg
Saved augmented image: augmented_blupus-14.jpeg
Saved augmented image: augmented_aria-12.jpeg
Saved augmented image: augmented_aria-14.jpeg
Saved augmented image: augmented_blupus-16.jpeg
Saved augmented image: augmented_blupus-03.jpeg
Saved augmented image: augmented_blupus-17.jpeg
Saved augmented image: augmented_aria-07.jpeg
Saved augmented image: augmented_aria-10.jpeg
Saved augmented image: augmented_blupus-13.jpeg
Saved augmented image: augmented_aria-02.jpeg
Saved augmented image: augmented_blupus-04.jpeg
Saved augmented image: augmented_aria-05.jpeg
Saved augmented image: augmented_blupus-07.jpeg
Saved augmented image: augmented_blupus-09.jpeg
Saved augmented image: augmented_blupus-06.jpeg
Saved augmented image: augmented_blupus-10.jpeg
Saved au

## Finetune a Model: Dreambooth Style

## Inference Time: Make me a Blupus!